<a href="https://colab.research.google.com/github/joserobertofox/datascience/blob/main/MVP2_DL_Vis%C3%A3o_Computacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP de Machine Learning & Analytics - Deep Learning - Visão Computacional**
## **Aluno: José Roberto Assis Silva**

## **1. Definição do Problema**

Este conjunto de dados foi criado com a finalidade de usá-lo em um drone e mapear/detectar doenças no campo de milho. Este conjunto de dados contém 4117 imagens de folhas doentes e saudáveis, distribuidas em 1550 doentes e 2567 saudáveis. Vamos criar um modelo que consiga mapear estas imagens e classificar.


In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.preprocessing import image
import os
from sklearn.model_selection import train_test_split
import shutil
import random
import math
import numpy as np
import zipfile
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from datetime import datetime
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
import sklearn.metrics as skm
import itertools
import numpy as np
import matplotlib.pyplot as plt